In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Cystic_Fibrosis"
cohort = "GSE129168"

# Input paths
in_trait_dir = "../../input/GEO/Cystic_Fibrosis"
in_cohort_dir = "../../input/GEO/Cystic_Fibrosis/GSE129168"

# Output paths
out_data_file = "../../output/preprocess/Cystic_Fibrosis/GSE129168.csv"
out_gene_data_file = "../../output/preprocess/Cystic_Fibrosis/gene_data/GSE129168.csv"
out_clinical_data_file = "../../output/preprocess/Cystic_Fibrosis/clinical_data/GSE129168.csv"
json_path = "../../output/preprocess/Cystic_Fibrosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"High-throughput screening for modulators of CFTR activity applying an organotypic functional assay based on genetically engineered Cystic Fibrosis disease-specific iPSCs"
!Series_summary	"Organotypic culture systems from disease-specific induced pluripotent stem cells (iPSCs) exhibit obvious advantages compared to immortalized cell lines and primary cell cultures but implementation of iPSC-based high throughput (HT) assays is still technically challenging. Here we demonstrate the development and conduction of an organotypic HT Cl-/I- exchange assay using Cystic Fibrosis (CF) disease-specific iPSCs. The introduction of a halide sensitive YFP variant enabled automated quantitative measurement of Cystic Fibrosis Transmembrane Conductance Regulator (CFTR) function in iPSC-derived intestinal epithelia. CFTR function was partially rescued by treatment with VX-770 and VX-809, and seamless gene correction of the p.Phe508del mutation resulted in full resto

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# This dataset contains gene expression data as it relates to CFTR function analysis
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Cystic Fibrosis): 
# We can infer this from row 2 which contains genotype information about CFTR mutations
trait_row = 2

# For age: Age is not explicitly provided and cannot be reliably inferred
age_row = None

# For gender: Gender is not specified in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert CFTR genotype to binary trait value for Cystic Fibrosis"""
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # CFTR p.Phe508del mutation indicates CF
    if 'p.Phe508del' in value and 'gene corrected' not in value:
        return 1  # Has CF
    else:
        return 0  # Does not have CF (CFTR WT or gene corrected)

def convert_age(value):
    """Placeholder function since age data is not available"""
    return None

def convert_gender(value):
    """Placeholder function since gender data is not available"""
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Initial filtering on usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Read the clinical data file
    clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
    if os.path.exists(clinical_data_path):
        clinical_data = pd.read_csv(clinical_data_path)
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the dataframe
        print("Preview of selected clinical features:")
        print(preview_df(selected_clinical_df))
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the clinical features dataframe
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    else:
        print(f"Clinical data file not found at {clinical_data_path}")


Clinical data file not found at ../../input/GEO/Cystic_Fibrosis/GSE129168/clinical_data.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 73
Header line: "ID_REF"	"GSM3701915"	"GSM3701916"	"GSM3701917"	"GSM3701918"	"GSM3701919"	"GSM3701920"	"GSM3701921"	"GSM3701922"	"GSM3701923"	"GSM3701924"	"GSM3701925"	"GSM3701926"	"GSM3701927"	"GSM3701928"	"GSM3701929"	"GSM3701930"	"GSM3701931"	"GSM3701932"	"GSM3701933"	"GSM3701934"	"GSM3701935"	"GSM3701936"	"GSM3701937"	"GSM3701938"	"GSM3701939"	"GSM3701940"	"GSM3701941"	"GSM3701942"	"GSM3701943"	"GSM3701944"	"GSM3701945"	"GSM3701946"
First data line: "A_23_P100001"	1884	1647	1959	3268	3346	3801	1817	1418	1572	3890	3638	4027	2250	1820	7073	8081	7426	2005	2163	2350	5476	4873	4716	2374	2490	2381	1399	1333	1287	818	984	901
Index(['A_23_P100001', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074',
       'A_23_P100127', 'A_23_P100141', 'A_23_P100189', 'A_23_P100196',
       'A_23_P100203', 'A_23_P100220', 'A_23_P100240', 'A_23_P10025',
       'A_23_P100292', 'A_23_P100315', 'A_23_P100326', 'A_23_P100344',
       'A_23_P100355', 'A_23_P100386', 'A_23_P100392', 'A_23_

### Step 4: Gene Identifier Review

In [5]:
# Based on the output, the gene identifiers in this dataset appear to be Agilent microarray probe IDs 
# (format A_23_Pxxxxxx), not human gene symbols. These identifiers need to be mapped to gene symbols
# for proper analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['A_23_P100001', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074', 'A_23_P100127'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_014848', 'NM_194272', 'NM_020371', 'NM_170589'], 'GB_ACC': ['NM_207446', 'NM_014848', 'NM_194272', 'NM_020371', 'NM_170589'], 'LOCUSLINK_ID': [400451.0, 9899.0, 348093.0, 57099.0, 57082.0], 'GENE_SYMBOL': ['FAM174B', 'SV2B', 'RBPMS2', 'AVEN', 'KNL1'], 'GENE_NAME': ['family with sequence similarity 174 member B', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein, mRNA processing factor 2', 'apoptosis and caspase activation inhibitor', 'kinetochore scaffold 1'], 'UNIGENE_ID': ['Hs.27373', 'Hs.21754', 'Hs.436518', 'Hs.555966', 'Hs.181855'], 'ENSEMBL_ID': ['ENST00000553393', nan, nan, 'ENST00000306730', 'ENST00000527044'], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': ['ref|NM_207446|ens|ENST00000553393|ens|ENST00000327355|ref|XR_93

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns in gene annotation for mapping
probe_column = 'ID'           # This is the probe identifier column that matches the gene expression index
gene_symbol_column = 'GENE_SYMBOL'  # This is the column with the gene symbols we want to map to

# Print relevant information for validation
print(f"Using probe IDs from column '{probe_column}' and gene symbols from '{gene_symbol_column}'")
print(f"First few probe IDs: {gene_annotation[probe_column][:5].tolist()}")
print(f"First few gene symbols: {gene_annotation[gene_symbol_column][:5].tolist()}")

# 2. Create a mapping dataframe with these two columns
mapping_df = get_gene_mapping(gene_annotation, probe_column, gene_symbol_column)
print(f"Created mapping dataframe with shape: {mapping_df.shape}")
print(f"Mapping preview: {preview_df(mapping_df)}")

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# This function handles the many-to-many mapping and proper distribution of expression values
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Generated gene expression data with shape: {gene_data.shape}")
print(f"First 5 gene symbols in the gene expression data: {gene_data.index[:5].tolist()}")


Using probe IDs from column 'ID' and gene symbols from 'GENE_SYMBOL'
First few probe IDs: ['A_23_P100001', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074', 'A_23_P100127']
First few gene symbols: ['FAM174B', 'SV2B', 'RBPMS2', 'AVEN', 'KNL1']
Created mapping dataframe with shape: (30331, 2)
Mapping preview: {'ID': ['A_23_P100001', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074', 'A_23_P100127'], 'Gene': ['FAM174B', 'SV2B', 'RBPMS2', 'AVEN', 'KNL1']}
Generated gene expression data with shape: (20520, 32)
First 5 gene symbols in the gene expression data: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-1']


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Extract clinical features directly from the matrix file
try:
    # Get the file paths for the matrix file to extract clinical data
    _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    
    # Get raw clinical data from the matrix file
    _, clinical_raw = get_background_and_clinical_data(matrix_file)
    
    # Verify clinical data structure
    print("Raw clinical data shape:", clinical_raw.shape)
    
    # Extract clinical features using the defined conversion functions
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_raw,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    print("Clinical features:")
    print(clinical_features)
    
    # Save clinical features to file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")
    
    # 3. Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
    print("Linked data preview (first 5 rows, first 5 columns):")
    print(linked_data.iloc[:5, :5])
    
    # 4. Handle missing values
    print("Missing values before handling:")
    print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
    if 'Age' in linked_data.columns:
        print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
    if 'Gender' in linked_data.columns:
        print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
    print(f"  Genes with >20% missing: {sum(linked_data.iloc[:, 1:].isna().mean() > 0.2)}")
    print(f"  Samples with >5% missing genes: {sum(linked_data.iloc[:, 1:].isna().mean(axis=1) > 0.05)}")
    
    cleaned_data = handle_missing_values(linked_data, trait)
    print(f"Data shape after handling missing values: {cleaned_data.shape}")
    
    # 5. Evaluate bias in trait and demographic features
    is_trait_biased = False
    if len(cleaned_data) > 0:
        trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
        is_trait_biased = trait_biased
    else:
        print("No data remains after handling missing values.")
        is_trait_biased = True
    
    # 6. Final validation and save
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=is_trait_biased, 
        df=cleaned_data,
        note="Dataset contains gene expression data comparing CFTR WT vs CFTR mutant (p.Phe508del) samples."
    )
    
    # 7. Save if usable
    if is_usable and len(cleaned_data) > 0:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        cleaned_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Data was determined to be unusable or empty and was not saved")
        
except Exception as e:
    print(f"Error processing data: {e}")
    # Handle the error case by still recording cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Mark as not available due to processing issues
        is_biased=True, 
        df=pd.DataFrame(),  # Empty dataframe
        note=f"Error processing data: {str(e)}"
    )
    print("Data was determined to be unusable and was not saved")

Normalized gene data shape: (20134, 32)


First few genes with their expression values after normalization:
          GSM3701915  GSM3701916  GSM3701917  GSM3701918  GSM3701919  \
Gene                                                                   
A1BG          4010.0      4608.0      3222.0       865.0       889.0   
A1BG-AS1       197.0       166.0       205.0       122.0       101.0   
A1CF             7.0         8.0         7.0      5318.0      5750.0   
A2M             11.0        10.0       255.0      1018.0      1230.0   
A2M-AS1        224.0       215.0       218.0       222.0       265.0   

          GSM3701920  GSM3701921  GSM3701922  GSM3701923  GSM3701924  ...  \
Gene                                                                  ...   
A1BG           810.0      5707.0      4568.0      3420.0       841.0  ...   
A1BG-AS1       155.0       147.0       179.0       178.0       163.0  ...   
A1CF          5530.0         6.0         6.0         7.0      5716.0  ...   
A2M           1378.0         5.0        10.

Normalized gene data saved to ../../output/preprocess/Cystic_Fibrosis/gene_data/GSE129168.csv
Raw clinical data shape: (4, 33)
Clinical features:
                 GSM3701915  GSM3701916  GSM3701917  GSM3701918  GSM3701919  \
Cystic_Fibrosis         1.0         1.0         1.0         1.0         1.0   

                 GSM3701920  GSM3701921  GSM3701922  GSM3701923  GSM3701924  \
Cystic_Fibrosis         1.0         0.0         0.0         0.0         0.0   

                 ...  GSM3701937  GSM3701938  GSM3701939  GSM3701940  \
Cystic_Fibrosis  ...         0.0         0.0         0.0         0.0   

                 GSM3701941  GSM3701942  GSM3701943  GSM3701944  GSM3701945  \
Cystic_Fibrosis         0.0         0.0         0.0         0.0         0.0   

                 GSM3701946  
Cystic_Fibrosis         0.0  

[1 rows x 32 columns]
Clinical features saved to ../../output/preprocess/Cystic_Fibrosis/clinical_data/GSE129168.csv
Linked data shape: (32, 20135)
Linked data preview (fi

Data shape after handling missing values: (32, 20135)
For the feature 'Cystic_Fibrosis', the least common label is '1.0' with 6 occurrences. This represents 18.75% of the dataset.
The distribution of the feature 'Cystic_Fibrosis' in this dataset is fine.



Linked data saved to ../../output/preprocess/Cystic_Fibrosis/GSE129168.csv
